In [ ]:
#!pip install ibapi


In [11]:
import datetime
import pytz

In [10]:
import sys
sys.path.append("/ibkr")

import pandas as pd
#from ibapi.order import Order
import passwords
from databaseClass import DB
from utils import generate_df_from_sql_file, generate_list_from_sql_file
from utils import DataProcessor, ModelBuilder, Predictor
import datetime
import pytz
pd.set_option('display.max_rows', None)

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:

def paper_trade(db, app, api_thread, contract, model, time_steps, look_ahead):
    def predict_price():
        # get the current time
        now = datetime.datetime.now()
        # get the time one minute ago
        one_minute_ago = now - datetime.timedelta(minutes=1)
        
        # query the database for all rows from the last minute
        new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun14 WHERE timestamp BETWEEN '{one_minute_ago}' AND '{now}'")
        
        if not new_data.empty:
            # Reshape data
#             new_data = new_data.pivot_table(index='timestamp', columns='tickType', values=['price', 'volume'])
#             new_data.columns = ['_'.join(map(str,i)) for i in new_data.columns]
#             new_data = new_data.resample('1S').agg({'price_1': 'last', 'price_2': 'last', 'price_4': 'last', 'volume_5': 'sum'}).ffill()
            
            # Process
            processor = DataProcessor(new_data) #do I need to pass this in the function
            processor.process_df() # Now I can access the data with processor.processed_df
            

            # Process and predict
#             preprocessor.df = new_data
#             preprocessor.process_df()
#             predictor = Predictor(model, preprocessor)
#             predictions = predictor.predict(look_ahead, time_steps)
#             price = predictor.rescale_prediction(predictions[-1])

        



            # Send order
            order = Contract()
            order.symbol = "ES"
            order.secType = "FUT"
            order.exchange = "CME"
            order.currency = "USD"
            order.lastTradeDateOrContractMonth = "202309"

            if price > new_data['price_4'][-1]:
                # Buy if the predicted price is higher than the current price
                app.placeOrder(app.nextOrderId(), order, Order(action="BUY", totalQuantity=1, orderType="LMT", lmtPrice=price))
            elif price < new_data['price_4'][-1]:
                # Sell if the predicted price is lower than the current price
                app.placeOrder(app.nextOrderId(), order, Order(action="SELL", totalQuantity=1, orderType="LMT", lmtPrice=price))

    while True:
        time.sleep(60)  # Wait for a minute
        predict_price()

if __name__ == "__main__":
    app, api_thread, contract = app_connect()
    # Load preprocessor and model
    preprocessor = DataProcessor(df=None)
    model = load_model('model_path')  # replace 'model_path' with the path to your model
    paper_trade(app, api_thread, contract, model, preprocessor, time_steps=10, look_ahead=1)

In [ ]:
def paper_trade(db, app, api_thread, contract, preprocessor, model_builder, predictor, time_steps, look_ahead):

    # Get current time
    now = datetime.datetime.now()

    # Get the time one minute ago
    one_minute_ago = now - datetime.timedelta(minutes=1)
    
    # Get the time five minutes ago
    five_minutes_ago = now - datetime.timedelta(minutes=5)
        
    # Query the database for all rows from the last minute
    new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun14 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")

    # Process new_data into the proper format for the model
    preprocessor.df = new_data
    preprocessor.process_df()
    # After preprocessing and filling, trim data to only last minute
    preprocessor.processed_df = preprocessor.processed_df.loc[one_minute_ago:]

    preprocessor.scale_shift_data(look_ahead)
    
    X, y = preprocessor.scaled_df, preprocessor.shifted_df['Close']
    X, y = preprocessor.create_dataset(X, y, time_steps)

    # Load the model
    model_builder.load_model("path_to_your_model")

    # Assign the model to the predictor
    predictor.model = model_builder.model

    # Predict the next 'look_ahead' steps
    predictions = predictor.predict(look_ahead, time_steps)

    # Rescale predictions back to the original scale
    rescaled_predictions = predictor.rescale_prediction(predictions)
    
    # Send order
    order = Contract()
    order.symbol = "ES"
    order.secType = "FUT"
    order.exchange = "CME"
    order.currency = "USD"
    order.lastTradeDateOrContractMonth = "202309"

    if price > new_data['price_4'][-1]:
        # Buy if the predicted price is higher than the current price
        app.placeOrder(app.nextOrderId(), order, Order(action="BUY", totalQuantity=1, orderType="LMT", lmtPrice=price))
    elif price < new_data['price_4'][-1]:
        # Sell if the predicted price is lower than the current price
        app.placeOrder(app.nextOrderId(), order, Order(action="SELL", totalQuantity=1, orderType="LMT", lmtPrice=price))

    while True:
        time.sleep(60)  # Wait for a minute
        predict_price()
        
def app_connect():
    db = DB(userName=userName, userPass=userPass, dataBaseName=dataBaseName, host=host, docker=False)
    app = App(db, tableName="tickdata_jun14")
    app.connect("127.0.0.1", 7497, 3)
    time.sleep(5) #use for paper trading
    contract = Contract()
    contract.symbol = "ES"
    contract.secType = "FUT"
    contract.exchange = "CME"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = "202309" # Please check the contract month

    # Start the socket in a thread
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()
    return app, api_thread, contract

if __name__ == "__main__":
    app, api_thread, contract = app_connect()
    # Load preprocessor and model
    preprocessor = DataProcessor(df=None)
    model = load_model('model_path')  # replace 'model_path' with the path to your model
    paper_trade(app, api_thread, contract, model, preprocessor, time_steps=10, look_ahead=1)

In [16]:
# now = datetime.datetime.now()
now = datetime.datetime.now(pytz.UTC)
one_minute_ago = now - datetime.timedelta(minutes=1)

In [14]:
now = datetime.datetime(2023, 6, 21, 21, 25, 20, 410, pytz.UTC)

In [17]:
now

datetime.datetime(2023, 6, 21, 22, 1, 24, 23257, tzinfo=<UTC>)

In [172]:
userName = passwords.userName
userPass = passwords.userPass
dataBaseName = passwords.dataBaseName
host = passwords.host

db = DB(userName=userName, userPass=userPass, dataBaseName=dataBaseName, host='ibkr_db', docker=True)

In [173]:
new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun14 WHERE timestamp BETWEEN '{one_minute_ago}' AND '{now}'")
#new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun13 LIMIT 100")

/ibkr/databaseClass.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=conn)


In [174]:
db.DBtoDF(f"SELECT * FROM tickdata_jun14").tail()

,ticktype,price,volume,timestamp
1197,2,4422.75,0,2023-06-14 23:22:46.663742
1198,5,0.00,5,2023-06-14 23:22:52.373177
1199,2,4423.00,0,2023-06-14 23:22:52.403540
1200,5,0.00,1,2023-06-14 23:22:54.175338
1201,2,4422.75,0,2023-06-14 23:22:54.206712


In [175]:
new_data.empty

False

In [176]:
processor = DataProcessor(new_data)

In [177]:
processor.process_df()

In [178]:
processor.processed_df.head()

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-14 22:55:44,NaN,4424.0,4424.00,4424.0,0.0
2023-06-14 22:55:45,4423.75,4424.0,4423.75,4424.0,24.0
2023-06-14 22:55:46,4423.75,4424.0,4423.75,4424.0,0.0
2023-06-14 22:55:47,4423.75,4424.0,4423.75,4424.0,0.0
2023-06-14 22:55:48,4423.75,4424.0,4423.75,4424.0,0.0


In [179]:
processor.scale_shift_data(look_ahead=5)

(MinMaxScaler(), MinMaxScaler())

In [181]:
processor.scaled_df.head()

,Open,High,Low,Close,Volume
timestamp,,,,,
2023-06-14 22:55:44,NaN,1.0,1.000000,1.0,0.0
2023-06-14 22:55:45,1.0,1.0,0.666667,1.0,1.0
2023-06-14 22:55:46,1.0,1.0,0.666667,1.0,0.0
2023-06-14 22:55:47,1.0,1.0,0.666667,1.0,0.0
2023-06-14 22:55:48,1.0,1.0,0.666667,1.0,0.0


In [182]:
new_data.head(20)

,ticktype,price,volume,timestamp
0,4,4424.00,0,2023-06-14 22:55:44.971996
1,5,0.00,10,2023-06-14 22:55:45.000595
2,5,0.00,10,2023-06-14 22:55:45.006522
3,1,4423.75,0,2023-06-14 22:55:45.016959
4,2,4424.00,0,2023-06-14 22:55:45.022909
5,5,0.00,4,2023-06-14 22:55:45.973196
6,4,4423.75,0,2023-06-14 22:55:51.881734
7,5,0.00,1,2023-06-14 22:55:51.908101
8,5,0.00,1,2023-06-14 22:55:51.914846
9,5,0.00,2,2023-06-14 22:55:54.285405


In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import datetime
import pandas as pd
from databaseClass import DB
import passwords
from utils import DataProcessor
from utils import ModelBuilder
from utils import Predictor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn
import pytz
from joblib import dump, load

userName = passwords.userName
userPass = passwords.userPass
dataBaseName = passwords.dataBaseName
host = passwords.host

class App(EWrapper, EClient):
    def __init__(self, db, tableName):
        EClient.__init__(self, self)
        self.db = db
        self.tableName = tableName

def paper_trade(app, api_thread, contract, preprocessor, model_builder, predictor, time_steps, look_ahead):

    # Get current time
    now = datetime.datetime.now(pytz.UTC)

    # Get the time one minute ago
    one_minute_ago = now - datetime.timedelta(minutes=1)
    
    # Get the time five minutes ago
    five_minutes_ago = now - datetime.timedelta(minutes=5)
        
    # Query the database for all rows from the last minute
    new_data = app.db.DBtoDF(f"SELECT * FROM tickdata_jun14 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")
    new_data['timestamp'] = new_data['timestamp'].dt.tz_localize('UTC')


    # Process new_data into the proper format for the model
    preprocessor.df = new_data
    preprocessor.process_df()
    # After preprocessing and filling, trim data to only last minute
    preprocessor.processed_df = preprocessor.processed_df.loc[one_minute_ago:]

    preprocessor.scale_shift_data(look_ahead)
    
    X, y = preprocessor.scaled_df, preprocessor.shifted_df['Close']
    X, y = preprocessor.create_dataset(X, y, time_steps)

    # Load the model
    model_builder.load_model("models/model.h5")

    # Assign the model to the predictor
    predictor.model = model_builder.model

    # Predict the next 'look_ahead' steps
    predictions = predictor.predict(look_ahead, time_steps)

    # Rescale predictions back to the original scale
    rescaled_predictions = predictor.rescale_prediction(predictions)
    
    # Fetch the last predicted value
    price = rescaled_predictions[-1][0]

    if price > new_data['price_4'][-1]:
        # Buy if the predicted price is higher than the current price
        #app.placeOrder(app.nextOrderId(), order, Order(action="BUY", totalQuantity=1, orderType="LMT", lmtPrice=price))
        print("buy")
    elif price < new_data['price_4'][-1]:
        # Sell if the predicted price is lower than the current price
        #app.placeOrder(app.nextOrderId(), order, Order(action="SELL", totalQuantity=1, orderType="LMT", lmtPrice=price))
        print("sell")

    while True:
        time.sleep(10)  # Wait for a second
        # predict_price()
        
def app_connect():
    db = DB(userName=userName, userPass=userPass, dataBaseName=dataBaseName, host=host, docker=False)
    app = App(db, tableName="tickdata_jun14")
    app.connect("127.0.0.1", 7497, 7)
    time.sleep(5) #use for paper trading
    contract = Contract()
    contract.symbol = "ES"
    contract.secType = "FUT"
    contract.exchange = "CME"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = "202309" # Please check the contract month

    # Start the socket in a thread
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()
    return app, api_thread, contract

if __name__ == "__main__":
    app, api_thread, contract = app_connect()
    preprocessor = DataProcessor(df=None)
    model_builder = ModelBuilder(n_features=5, time_steps=60)
    predictor = Predictor(model=None, preprocessor=preprocessor)
    paper_trade(app, api_thread, contract, preprocessor, model_builder, predictor, time_steps=60, look_ahead=5)

In [226]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import datetime
import pandas as pd
from databaseClass import DB
import passwords
from utils import DataProcessor
from utils import ModelBuilder
from utils import Predictor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn
import pytz
from joblib import dump, load

userName = passwords.userName
userPass = passwords.userPass
dataBaseName = passwords.dataBaseName
host = passwords.host

class App(EWrapper, EClient):
    def __init__(self, db, tableName):
        EClient.__init__(self, self)
        self.db = db
        self.tableName = tableName

In [234]:
# Get current time
now = datetime.datetime.now(pytz.UTC)

# Get the time one minute ago
one_minute_ago = now - datetime.timedelta(minutes=1)

# Get the time five minutes ago
five_minutes_ago = now - datetime.timedelta(minutes=5)

new_data = db.DBtoDF(f"SELECT * FROM tickdata_jun15 WHERE timestamp BETWEEN '{five_minutes_ago}' AND '{now}'")

new_data['timestamp'] = new_data['timestamp'].dt.tz_localize('UTC')

preprocessor = DataProcessor(df=None)
model_builder = ModelBuilder(n_features=5, time_steps=60)
predictor = Predictor(model=None, preprocessor=preprocessor)
preprocessor.df = new_data
preprocessor.process_df()
preprocessor.scale_shift_data(look_ahead=5)
X, y = preprocessor.scaled_df, preprocessor.shifted_df['Close']
X, y = preprocessor.create_dataset(X, y, time_steps=5)
model_builder.load_model("../models/model.h5")
predictor.model = model_builder.model
predictions = predictor.predict(look_ahead=5, time_steps=60)
rescaled_predictions = predictor.rescale_prediction(predictions)
rescaled_predictions[-1][0]

/ibkr/databaseClass.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con=conn)


8/8 [==============================] - 0s 5ms/step


4454.3774

In [233]:

# new_data = db.DBtoDF("SELECT * FROM tickdata_jun13")

In [ ]:




# Fetch the last predicted value
price = rescaled_predictions[-1][0]

if price > new_data['price_4'][-1]:
    # Buy if the predicted price is higher than the current price
    #app.placeOrder(app.nextOrderId(), order, Order(action="BUY", totalQuantity=1, orderType="LMT", lmtPrice=price))
    print("buy")
elif price < new_data['price_4'][-1]:
    # Sell if the predicted price is lower than the current price
    #app.placeOrder(app.nextOrderId(), order, Order(action="SELL", totalQuantity=1, orderType="LMT", lmtPrice=price))
    print("sell")
